In [1]:
import json

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import requests
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

In [2]:
url = "http://cocl.us/Geospatial_data"
toronto_data = pd.read_csv(url)
toronto_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
# load neighborhood and borough data from other notebook
neighborhoods = pd.read_csv('toronto_neighborhoods.csv')
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park / Ontario Provincial Government,"Queen's Park, Ontario Provincial Government"


merge the 2 dataframes together on the postal codes. Drop the second postal code column after the merge

In [4]:
neighborhoods = pd.merge(left = neighborhoods, right = toronto_data, left_on ='PostalCode', right_on = 'Postal Code')
neighborhoods.drop(columns = 'PostalCode', axis = 1, inplace = True)
neighborhoods.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Queen's Park / Ontario Provincial Government,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


In [5]:
print(f'There are {neighborhoods.Borough.nunique()} Boroughs and {neighborhoods.Neighborhood.nunique()} Neighborhoods')

There are 15 Boroughs and 100 Neighborhoods


Use Geopy to get the location for Toronto. An instance of geocoder needs to be defined

In [6]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


create a map of Toronto and mark all of the neighborhood locations

In [7]:
# create map of Toront using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

Focus only on the boroughs that contain 'Toronto'

In [8]:
toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
1,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937
2,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
3,East Toronto,The Beaches,M4E,43.676357,-79.293031
4,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306


In [9]:
print(f'Now there are {toronto_data.Borough.nunique()} Boroughs and {toronto_data.Neighborhood.nunique()} Neighborhoods')

Now there are 7 Boroughs and 39 Neighborhoods


look at a less populated neighborhood map

In [10]:
# create map of Toront using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

foursquare api access

In [25]:
CLIENT_ID = 'xxxxxxCLIENTIDxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxCLIENTSECRETxxxxxxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxxxxxCLIENTIDxxxxxx
CLIENT_SECRET:xxxxxxCLIENTSECRETxxxxxxxx


function to get nearby venues of the neighborhoods

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
limit = 100 # limit search to 100 results
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Regent Park,  Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond,  Adelaide,  King
Dufferin,  Dovercourt Village
The Danforth East
Harbourfront East,  Union Station,  Toronto Islands
Little Portugal,  Trinity
The Danforth West,  Riverdale
Toronto Dominion Centre,  Design Exchange
Brockton,  Parkdale Village,  Exhibition Place
India Bazaar,  The Beaches West
Commerce Court,  Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park,  The Junction South
North Toronto West
The Annex,  North Midtown,  Yorkville
Parkdale,  Roncesvalles
Davisville
University of Toronto,  Harbord
Runnymede,  Swansea
Moore Park,  Summerhill East
Kensington Market,  Chinatown,  Grange Park
Summerhill West,  Rathnelly,  South Hill,  Forest Hill SE,  Deer Park
CN Tower,  King and Spadina,  Railway Lands,  Harbourfront West,  Bathurst Quay,  South Niagara,  Island airport
Rosedale
Enclave of M5E
St. James

In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

create new dataframe to list the most common venues in each neighborhood

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Cheese Shop,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Eastern European Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Breakfast Spot,Café,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Performing Arts Venue
2,"CN Tower, King and Spadina, Railway Lands, ...",Airport Service,Airport Lounge,Boat or Ferry,Coffee Shop,Harbor / Marina,Boutique,Airport Terminal,Bar,Airport Gate,Airport Food Court
3,Central Bay Street,Coffee Shop,Italian Restaurant,Burger Joint,Japanese Restaurant,Ice Cream Shop,Sandwich Place,Salad Place,Department Store,Gym / Fitness Center,Middle Eastern Restaurant
4,Christie,Grocery Store,Café,Park,Nightclub,Baby Store,Candy Store,Diner,Italian Restaurant,Restaurant,Gas Station


In [18]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,2,Coffee Shop,Bakery,Pub,Park,Restaurant,Theater,Café,Mexican Restaurant,Chocolate Shop,Dessert Shop
1,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Italian Restaurant,Bubble Tea Shop,Pizza Place,Bookstore
2,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Beer Bar,Clothing Store,Bakery,Cosmetics Shop,Diner
3,East Toronto,The Beaches,M4E,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
4,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306,2,Coffee Shop,Café,Cheese Shop,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Eastern European Restaurant


In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,The Beaches,0,Trail,Pub,Health Food Store,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Women's Store


### Cluster 2

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,1,Garden,Women's Store,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### Cluster 3

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",2,Coffee Shop,Bakery,Pub,Park,Restaurant,Theater,Café,Mexican Restaurant,Chocolate Shop,Dessert Shop
1,"Garden District, Ryerson",2,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Italian Restaurant,Bubble Tea Shop,Pizza Place,Bookstore
2,St. James Town,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Beer Bar,Clothing Store,Bakery,Cosmetics Shop,Diner
4,Berczy Park,2,Coffee Shop,Café,Cheese Shop,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Eastern European Restaurant
5,Central Bay Street,2,Coffee Shop,Italian Restaurant,Burger Joint,Japanese Restaurant,Ice Cream Shop,Sandwich Place,Salad Place,Department Store,Gym / Fitness Center,Middle Eastern Restaurant
6,Christie,2,Grocery Store,Café,Park,Nightclub,Baby Store,Candy Store,Diner,Italian Restaurant,Restaurant,Gas Station
7,"Richmond, Adelaide, King",2,Coffee Shop,Restaurant,Café,Thai Restaurant,Steakhouse,Bar,Clothing Store,Bakery,Breakfast Spot,Concert Hall
8,"Dufferin, Dovercourt Village",2,Bakery,Pharmacy,Pizza Place,Grocery Store,Gym / Fitness Center,Fast Food Restaurant,Middle Eastern Restaurant,Music Venue,Pool,Café
10,"Harbourfront East, Union Station, Toronto Is...",2,Coffee Shop,Aquarium,Hotel,Italian Restaurant,Café,Fried Chicken Joint,Scenic Lookout,Sporting Goods Shop,Restaurant,Brewery
11,"Little Portugal, Trinity",2,Bar,Coffee Shop,Restaurant,Asian Restaurant,Pizza Place,Men's Store,Wine Bar,Vietnamese Restaurant,Café,Juice Bar


### Cluster 4

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,The Danforth East,3,Park,Convenience Store,Metro Station,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
18,Lawrence Park,3,Park,Gym / Fitness Center,Swim School,Bus Line,Women's Store,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
21,Forest Hill North & West,3,Park,Jewelry Store,Trail,Sushi Restaurant,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
29,"Moore Park, Summerhill East",3,Park,Gym,Tennis Court,Playground,Women's Store,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center
33,Rosedale,3,Park,Playground,Trail,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
